In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision import utils

is_cuda = torch.cuda.is_available()

In [2]:
'''
set the hyperparameters of the model:
including batch size for test and training, number of epoch, and smoothing error
'''

TRAIN_BATCH_SIZE = 200
TEST_BATCH_SIZE = 10000
FULL_TRAIN_BATCH_SIZE = 50000
EPOCH = 50
EPOCH_2 = 100
TINY_ERROR = 1e-15
DATA_PATH = "/floyd/input/skripsi_datasets_2/"

#check whether the computer has GPU
cuda = torch.device('cuda')

In [3]:
'''
Load data for training and testing
CIFAR-10:
50000 training, 10000 testing
there are 2 types: RGB and grayscale. RGB dataset is used for feature engineering
The RGB and Grayscale dataset are normalized using mean equals to 0.5 (all of 3 channels for RGB) 
and SD equals to 0.5 (all of 3 channels for RGB).
The data then load to the loader with batch size set in the previous cell
There are additional data loader to test the model with all of the data train (batch size 50000).
'''


#../../home/Data/mnist
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        ),
    ),
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
)

train_loader_2 = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        ),
    ),
    batch_size= FULL_TRAIN_BATCH_SIZE,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=False,
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        ),
    ),
    batch_size=TEST_BATCH_SIZE,
    shuffle=True,
)




#../../home/Data/mnist
train_loader_gs = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Grayscale(num_output_channels=1),
             transforms.ToTensor(),
             transforms.Normalize((0.5,), (0.5,))]
        ),
    ),
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
)

train_loader_gs_2 = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Grayscale(num_output_channels=1),
             transforms.ToTensor(),
             transforms.Normalize((0.5,), (0.5,))]
        ),
    ),
    batch_size=FULL_TRAIN_BATCH_SIZE,
    shuffle=True,
)

test_loader_gs = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "/Downloads/",
        train=False,
        download=True,
        transform=transforms.Compose(
            [transforms.Grayscale(num_output_channels=1),
             transforms.ToTensor(),
             transforms.Normalize((0.5,), (0.5,))]
        ),
    ),
    batch_size=TEST_BATCH_SIZE,
    shuffle=True,
)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

'''
Param: prediction of the model and ground truth
return: accuracy, sensitivity of each label, specificity of each label, and f1 score of each label
accuracy is computed by compared the number of the same value on prediction and ground truth
specificity and sensitivity is computed by looking up to the confusion matrix of each label
sensitivity =  tp / (tp + fn) 
specificity = tn / (tn + fp)
f1 score computed by: precision / recall
'''
def compute_metric(cprediction, ctarget):
    if torch.cuda.is_available():
        prediction = cprediction.cpu()
        target = ctarget.cpu()
    
    accuracy = accuracy_score(prediction, target)
    f1_sc = f1_score(prediction, target, labels=[i for i in range(10)], average=None)
    confusion_matrix = multilabel_confusion_matrix(prediction, target, labels=[i for i in range(10)])
    specificity = np.zeros(10)
    sensitivity = np.zeros(10) 
    
    for i in range(confusion_matrix.shape[0]):
        tn, fp, fn, tp = confusion_matrix[i].ravel()
        sensitivity[i] = tp / (tp + fn + 1e-4)
        specificity[i] = tn / (tn + fp + 1e-4)
    
    return accuracy, sensitivity, specificity, f1_sc

'''
parameter:
model, train loader, optimizer, loss function, and the current epoch
- this method trains a model for one epoch
- the model is set to train to unfreeze the weight parameter
- iterate over mini batch
- doing forward propagation
- compute the loss
- find the gradient w.r.t each of the parameter
- perform stochastic gradient descent
- keep the prediction and the ground truth, and compute the metric using method compute_metric
- if the epoch is a multiple of 5, output the metric score
- if the GPU exist, then train the model with GPU (cuda interface)
'''
def train_model(model, train_loader, optimizer, criterion, epoch):
    model = model.train()
    list_output = None
    list_target = None
    
    for batch_idx, (data, target) in enumerate(train_loader):
        if is_cuda:
            data = data.cuda(cuda)
            target = target.cuda(cuda)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        if list_output is None:
            list_output = output
        else:
            list_output = torch.cat((list_output, output), 0)
        
        if list_target is None:
            list_target = target
        else:
            list_target = torch.cat((list_target, target), 0)
        
        loss = criterion(output + TINY_ERROR, target)
        loss.backward()
        optimizer.step()
    
    
    _, prediction = torch.max(list_output, 1)
    correct = (prediction == list_target).squeeze()
    accuracy, sensitivity, specificity, f1_score = compute_metric(prediction, list_target)
    
    if (epoch % 5 == 0) :
        print('Epoch: {}\n'.format(epoch))
        print('Accuracy: {:.4}\n'.format(accuracy))
        print('Sensitivity for each class:\n')
        for i in range(sensitivity.shape[0]):
            print('    class {}: {:.4}\n'.format(i, sensitivity[i]))
        print('Specificity for each class:\n')

        for i in range(specificity.shape[0]):
            print('    class {}: {:.4}\n'.format(i, specificity[i]))

        print('f1-score for each class:\n')

        count = 0
        for f1 in (f1_score):
            print('    class {}: {:.4}\n'.format(count, f1))
            count += 1
            
        print()

In [5]:
'''
The architecture of original fully connected layer
The input size is the dimension of the input (grayscale CIFAR) which is 32x32
'''
INPUT_SIZE = 32 * 32

class fully_connected(nn.Module):
    def __init__(self):
        super(fully_connected, self).__init__()
        self.linear1 = nn.Linear(INPUT_SIZE, 1000)
        self.linear2 = nn.Linear(1000, 1000)
        self.linear3 = nn.Linear(1000, 10)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.view(-1, INPUT_SIZE)
        x = self.relu1(self.linear1(x))
        x = self.relu2(self.linear2(x))
        x = self.linear3(x)
        x = self.softmax(x)
        
        return x

In [6]:
'''
Train the original model with SGD 
using cross entropy loss
'''
model_fc = fully_connected()

if is_cuda:
    model_fc = model_fc.cuda(cuda)
    
criterion_fc = nn.CrossEntropyLoss()
optimizer_fc = optim.SGD(model_fc.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    train_model(model_fc, train_loader_gs, optimizer_fc, criterion_fc, epoch)

Epoch: 0

Accuracy: 0.1379

Sensitivity for each class:

    class 0: 0.1951

    class 1: 0.1041

    class 2: 0.1209

    class 3: 0.0

    class 4: 0.1254

    class 5: 0.1913

    class 6: 0.1053

    class 7: 0.1007

    class 8: 0.0

    class 9: 0.1681

Specificity for each class:

    class 0: 0.9046

    class 1: 0.9004

    class 2: 0.9064

    class 3: 0.9

    class 4: 0.9058

    class 5: 0.9062

    class 6: 0.9007

    class 7: 0.9

    class 8: 0.9

    class 9: 0.9229

f1-score for each class:

    class 0: 0.1232

    class 1: 0.1027

    class 2: 0.1692

    class 3: 0.0

    class 4: 0.1631

    class 5: 0.1481

    class 6: 0.1132

    class 7: 0.01061

    class 8: 0.0

    class 9: 0.2406


Epoch: 5

Accuracy: 0.1983

Sensitivity for each class:

    class 0: 0.216

    class 1: 0.2315

    class 2: 0.1463

    class 3: 0.2667

    class 4: 0.2179

    class 5: 0.2372

    class 6: 0.1822

    class 7: 0.2895

    class 8: 0.4096

    class 9: 0.1787

Specificity

In [7]:
'''
parameter: model and test loader
method to compute general loss
set the model to eval mode to freeze the weight parameter
calling this method will provide the model to compute the gradient for each of the computation
so it will reduce the usage of memory

use cuda interface to utilize GPU if the GPU exist
output the metric result
'''

def test_model(model, test_loader):
    model = model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if is_cuda:
                data = data.cuda(cuda)
                target = target.cuda(cuda)
            
            output = model(data)
            _, prediction = torch.max(output, 1)
            correct = (prediction == target).squeeze()
    
            accuracy, sensitivity, specificity, f1_score = compute_metric(prediction, target)
            
            print('Accuracy: {:.4}\n'.format(accuracy))
            print('Sensitivity for each class:\n')
            for i in range(sensitivity.shape[0]):
                print('    class {}: {:.4}\n'.format(i, sensitivity[i]))
            print('Specificity for each class:\n')

            for i in range(specificity.shape[0]):
                print('    class {}: {:.4}\n'.format(i, specificity[i]))

            print('f1-score for each class:\n')

            count = 0
            for f1 in f1_score:
                print('    class {}: {:.4}\n'.format(count, f1))
                count += 1
            print()

In [8]:
#save and test the model

file_location_model_fc = "model_fc.pt"
torch.save(model_fc.state_dict(), file_location_model_fc)

test_model(model_fc, test_loader_gs)
test_model(model_fc, train_loader_gs_2)

Accuracy: 0.2967

Sensitivity for each class:

    class 0: 0.3087

    class 1: 0.2855

    class 2: 0.0

    class 3: 0.0

    class 4: 0.2832

    class 5: 0.2919

    class 6: 0.2749

    class 7: 0.2445

    class 8: 0.3269

    class 9: 0.3512

Specificity for each class:

    class 0: 0.9268

    class 1: 0.9249

    class 2: 0.9

    class 3: 0.9

    class 4: 0.9195

    class 5: 0.9281

    class 6: 0.9206

    class 7: 0.9256

    class 8: 0.9398

    class 9: 0.9403

f1-score for each class:

    class 0: 0.3285

    class 1: 0.3095

    class 2: 0.0

    class 3: 0.0

    class 4: 0.278

    class 5: 0.3275

    class 6: 0.2822

    class 7: 0.2938

    class 8: 0.3915

    class 9: 0.4077


Accuracy: 0.2918

Sensitivity for each class:

    class 0: 0.299

    class 1: 0.2855

    class 2: 0.0

    class 3: 0.0

    class 4: 0.2683

    class 5: 0.2948

    class 6: 0.2625

    class 7: 0.2467

    class 8: 0.3189

    class 9: 0.3472

Specificity for each class:

    cla

In [9]:
'''
The architecture of feature engineering model
the input change since the input is in RGB form
'''

INPUT_SIZE_2 = 3 * 32 * 32


class feature_engineering_fully_connected(nn.Module):
    def __init__(self):
        super(feature_engineering_fully_connected, self).__init__()
        self.linear1 = nn.Linear(INPUT_SIZE_2, 1000)
        self.linear2 = nn.Linear(1000, 1000)
        self.linear3 = nn.Linear(1000, 10)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.view(-1, INPUT_SIZE_2)
        x = self.relu1(self.linear1(x))
        x = self.relu2(self.linear2(x))
        x = self.linear3(x)
        x = self.softmax(x)
        
        return x    


In [10]:
'''
Train the feature engineering model with SGD and cross entropy loss
Use cuda if the GPU exist
'''
model_fc_2 = feature_engineering_fully_connected()

if is_cuda:
    model_fc_2 = model_fc_2.cuda(cuda)
    
criterion_fc_2 = nn.CrossEntropyLoss()
optimizer_fc_2 = optim.SGD(model_fc_2.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    train_model(model_fc_2, train_loader, optimizer_fc_2, criterion_fc_2, epoch)

Epoch: 0

Accuracy: 0.1438

Sensitivity for each class:

    class 0: 0.2776

    class 1: 0.2121

    class 2: 0.1023

    class 3: 0.1264

    class 4: 0.1274

    class 5: 0.0

    class 6: 0.07883

    class 7: 0.09207

    class 8: 0.1582

    class 9: 0.1858

Specificity for each class:

    class 0: 0.9154

    class 1: 0.9011

    class 2: 0.9

    class 3: 0.9198

    class 4: 0.9008

    class 5: 0.9

    class 6: 0.8953

    class 7: 0.8999

    class 8: 0.9009

    class 9: 0.9252

f1-score for each class:

    class 0: 0.2467

    class 1: 0.03822

    class 2: 0.02401

    class 3: 0.2049

    class 4: 0.05662

    class 5: 0.0

    class 6: 0.1016

    class 7: 0.02697

    class 8: 0.04016

    class 9: 0.258


Epoch: 5

Accuracy: 0.2062

Sensitivity for each class:

    class 0: 0.2512

    class 1: 0.0

    class 2: 0.0

    class 3: 0.1947

    class 4: 0.1855

    class 5: 0.6909

    class 6: 0.1631

    class 7: 0.5

    class 8: 0.4452

    class 9: 0.2146

Speci

In [11]:
#save and test the model
file_location_model_fc_2 = "model_fc_2.pt"
torch.save(model_fc_2.state_dict(), file_location_model_fc_2)

test_model(model_fc_2, test_loader)
test_model(model_fc_2, train_loader_2)

Accuracy: 0.3928

Sensitivity for each class:

    class 0: 0.4651

    class 1: 0.4085

    class 2: 0.0

    class 3: 0.0

    class 4: 0.3489

    class 5: 0.3455

    class 6: 0.3417

    class 7: 0.3696

    class 8: 0.4559

    class 9: 0.4442

Specificity for each class:

    class 0: 0.9394

    class 1: 0.9423

    class 2: 0.9

    class 3: 0.9

    class 4: 0.9339

    class 5: 0.9327

    class 6: 0.954

    class 7: 0.9366

    class 8: 0.9533

    class 9: 0.9435

f1-score for each class:

    class 0: 0.459

    class 1: 0.4468

    class 2: 0.0

    class 3: 0.0

    class 4: 0.3803

    class 5: 0.3733

    class 6: 0.4416

    class 7: 0.4026

    class 8: 0.5158

    class 9: 0.4696


Accuracy: 0.3986

Sensitivity for each class:

    class 0: 0.4587

    class 1: 0.4115

    class 2: 0.0

    class 3: 0.0

    class 4: 0.3357

    class 5: 0.359

    class 6: 0.3453

    class 7: 0.3788

    class 8: 0.4818

    class 9: 0.4534

Specificity for each class:

    clas

In [12]:
'''
The architecture with additional layers
Adding a layer with 500 neurons
Using ReLU as its activation function
input size is same with the original model
'''

INPUT_SIZE = 32 * 32

class improved_fully_connected(nn.Module):
    def __init__(self):
        super(improved_fully_connected, self).__init__()
        self.linear1 = nn.Linear(INPUT_SIZE, 1000)
        self.linear2 = nn.Linear(1000, 1000)
        self.linear3 = nn.Linear(1000, 500)
        self.linear4 = nn.Linear(500, 10)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.view(-1, INPUT_SIZE)
        x = self.relu1(self.linear1(x))
        x = self.relu2(self.linear2(x))
        x = self.relu3(self.linear3(x))
        x = self.linear4(x)
        x = self.softmax(x)
        
        return x

In [13]:
'''
Train the improved model with SGD and cross entropy loss function
use cuda interface if GPU exist
'''
model_fc_imp = improved_fully_connected()

if is_cuda:
    model_fc_imp = model_fc_imp.cuda(cuda)
    
criterion_fc_imp = nn.CrossEntropyLoss()
optimizer_fc_imp = optim.SGD(model_fc_imp.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    train_model(model_fc_imp, train_loader_gs, optimizer_fc_imp, criterion_fc_imp, epoch)

Epoch: 0

Accuracy: 0.0969

Sensitivity for each class:

    class 0: 0.25

    class 1: 0.0

    class 2: 0.01786

    class 3: 0.05198

    class 4: 0.06029

    class 5: 0.104

    class 6: 0.0

    class 7: 0.0

    class 8: 0.2222

    class 9: 0.2

Specificity for each class:

    class 0: 0.9

    class 1: 0.9

    class 2: 0.8994

    class 3: 0.8996

    class 4: 0.8936

    class 5: 0.9218

    class 6: 0.9

    class 7: 0.9

    class 8: 0.9

    class 9: 0.9

f1-score for each class:

    class 0: 0.0003997

    class 1: 0.0

    class 2: 0.002249

    class 3: 0.007772

    class 4: 0.07006

    class 5: 0.186

    class 6: 0.0

    class 7: 0.0

    class 8: 0.0007986

    class 9: 0.0003996


Epoch: 5

Accuracy: 0.1674

Sensitivity for each class:

    class 0: 0.3275

    class 1: 0.1346

    class 2: 0.1818

    class 3: 0.1098

    class 4: 0.06445

    class 5: 0.123

    class 6: 0.1053

    class 7: 0.0

    class 8: 0.3547

    class 9: 0.2604

Specificity for eac

In [36]:
#save and test the model
file_location_model_fc_imp = "model_fc_imp.pt"
torch.save(model_fc_imp.state_dict(), file_location_model_fc_imp)

test_model(model_fc_imp, test_loader_gs)
test_model(model_fc_imp, train_loader_gs_2)

Accuracy: 0.2627

Sensitivity for each class:

    class 0: 0.275

    class 1: 0.2329

    class 2: 0.0

    class 3: 0.0

    class 4: 0.2338

    class 5: 0.2379

    class 6: 0.2272

    class 7: 0.0

    class 8: 0.3178

    class 9: 0.2899

Specificity for each class:

    class 0: 0.9297

    class 1: 0.9178

    class 2: 0.9

    class 3: 0.9

    class 4: 0.9175

    class 5: 0.9291

    class 6: 0.9162

    class 7: 0.9

    class 8: 0.9397

    class 9: 0.9418

f1-score for each class:

    class 0: 0.3256

    class 1: 0.2522

    class 2: 0.0

    class 3: 0.0

    class 4: 0.2506

    class 5: 0.3022

    class 6: 0.2407

    class 7: 0.0

    class 8: 0.3855

    class 9: 0.373


Accuracy: 0.2573

Sensitivity for each class:

    class 0: 0.2569

    class 1: 0.2352

    class 2: 0.0

    class 3: 0.0

    class 4: 0.2153

    class 5: 0.2456

    class 6: 0.2244

    class 7: 0.0

    class 8: 0.3047

    class 9: 0.2941

Specificity for each class:

    class 0: 0.9251

In [16]:
'''
train hyperparameter tuning model by change the epoch (epoch equals to 100)
change the optimator with Adam optimator 
set learning rate equals to 0.0002
use cuda interface if GPU exist
'''
model_fc_ht = fully_connected()
lr = 0.0002

if is_cuda:
    model_fc_ht = model_fc_ht.cuda(cuda)
    
criterion_fc_ht = nn.CrossEntropyLoss()
optimizer_fc_ht = optim.Adam(model_fc_ht.parameters(), lr)

for epoch in range(EPOCH_2):
    train_model(model_fc_ht, train_loader_gs, optimizer_fc_ht, criterion_fc_ht, epoch)

Epoch: 0

Accuracy: 0.2835

Sensitivity for each class:

    class 0: 0.2915

    class 1: 0.2763

    class 2: 0.2561

    class 3: 0.2038

    class 4: 0.2512

    class 5: 0.2976

    class 6: 0.2448

    class 7: 0.2969

    class 8: 0.303

    class 9: 0.3093

Specificity for each class:

    class 0: 0.9223

    class 1: 0.9237

    class 2: 0.908

    class 3: 0.9004

    class 4: 0.9149

    class 5: 0.9255

    class 6: 0.9215

    class 7: 0.9214

    class 8: 0.9338

    class 9: 0.9371

f1-score for each class:

    class 0: 0.2974

    class 1: 0.2996

    class 2: 0.1678

    class 3: 0.0165

    class 4: 0.2379

    class 5: 0.3175

    class 6: 0.2759

    class 7: 0.2938

    class 8: 0.3562

    class 9: 0.3717


Epoch: 5

Accuracy: 0.4125

Sensitivity for each class:

    class 0: 0.4276

    class 1: 0.4754

    class 2: 0.3588

    class 3: 0.3363

    class 4: 0.3346

    class 5: 0.4067

    class 6: 0.3736

    class 7: 0.4643

    class 8: 0.4533

    class 9: 

In [17]:
#save and test the model
file_location_model_fc_ht = "model_fc_ht.pt"
torch.save(model_fc_ht.state_dict(), file_location_model_fc_ht)

test_model(model_fc_ht, test_loader_gs)
test_model(model_fc_ht, train_loader_gs_2)

Accuracy: 0.461

Sensitivity for each class:

    class 0: 0.537

    class 1: 0.5602

    class 2: 0.388

    class 3: 0.318

    class 4: 0.3918

    class 5: 0.3713

    class 6: 0.4454

    class 7: 0.5496

    class 8: 0.5633

    class 9: 0.4871

Specificity for each class:

    class 0: 0.9413

    class 1: 0.9449

    class 2: 0.9315

    class 3: 0.9191

    class 4: 0.9313

    class 5: 0.9355

    class 6: 0.9443

    class 7: 0.946

    class 8: 0.9548

    class 9: 0.953

f1-score for each class:

    class 0: 0.4979

    class 1: 0.5273

    class 2: 0.3855

    class 3: 0.2837

    class 4: 0.3858

    class 5: 0.3985

    class 6: 0.4738

    class 7: 0.529

    class 8: 0.5792

    class 9: 0.5324


Accuracy: 0.7356

Sensitivity for each class:

    class 0: 0.7912

    class 1: 0.8082

    class 2: 0.6951

    class 3: 0.7097

    class 4: 0.6953

    class 5: 0.6738

    class 6: 0.7089

    class 7: 0.7913

    class 8: 0.7644

    class 9: 0.7255

Specificity for e

In [22]:
'''
The architecture of the original CNN model
flatten convolutional size means the flatten size of the input after passed the 
convolutional layers.
'''
FLATTEN_CONVOLUTION_SIZE = 8 * 8 * 50

class convolutional_layer(nn.Module):
    def __init__(self):
        super(convolutional_layer, self).__init__()
        self.convolution1 = nn.Conv2d(3, 25, 3, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.convolution2 = nn.Conv2d(25, 50, 3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.linear1 = nn.Linear(FLATTEN_CONVOLUTION_SIZE, 100)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.convolution1(x)
        x = self.maxpool1(x)
        x = self.convolution2(x)
        x = self.maxpool2(x)
        x = x.view(-1, FLATTEN_CONVOLUTION_SIZE)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        
        return x

In [23]:
'''
Train the model using SGD with lr equals to 0.001 and momentum equals to 0.9
The loss function is cross entropy loss function
'''

model_cnn =  convolutional_layer()

if is_cuda:
    model_cnn = model_cnn.cuda(cuda)
  
criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.SGD(model_cnn.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    train_model(model_cnn, train_loader, optimizer_cnn, criterion_cnn, epoch)

Epoch: 0

Accuracy: 0.1042

Sensitivity for each class:

    class 0: 0.1223

    class 1: 0.1001

    class 2: 0.02632

    class 3: 0.0

    class 4: 0.0

    class 5: 0.1751

    class 6: 0.0

    class 7: 0.1853

    class 8: 0.0

    class 9: 0.1667

Specificity for each class:

    class 0: 0.9008

    class 1: 0.9014

    class 2: 0.8999

    class 3: 0.9

    class 4: 0.9

    class 5: 0.903

    class 6: 0.9

    class 7: 0.9004

    class 8: 0.9

    class 9: 0.9

f1-score for each class:

    class 0: 0.06429

    class 1: 0.1806

    class 2: 0.000788

    class 3: 0.0

    class 4: 0.0

    class 5: 0.09795

    class 6: 0.0

    class 7: 0.01825

    class 8: 0.0

    class 9: 0.0003995


Epoch: 5

Accuracy: 0.2238

Sensitivity for each class:

    class 0: 0.2641

    class 1: 0.2113

    class 2: 0.3333

    class 3: 0.1888

    class 4: 0.2418

    class 5: 0.3321

    class 6: 0.1708

    class 7: 0.2758

    class 8: 0.4307

    class 9: 0.2215

Specificity for each 

In [24]:
#save and test the model
file_location_model_cnn = "model_cnn.pt"
torch.save(model_cnn.state_dict(), file_location_model_cnn)

test_model(model_cnn, test_loader)
test_model(model_cnn, train_loader_2)

Accuracy: 0.4918

Sensitivity for each class:

    class 0: 0.5746

    class 1: 0.5776

    class 2: 0.3829

    class 3: 0.3467

    class 4: 0.4418

    class 5: 0.4268

    class 6: 0.5004

    class 7: 0.5419

    class 8: 0.5654

    class 9: 0.5231

Specificity for each class:

    class 0: 0.9449

    class 1: 0.9541

    class 2: 0.9304

    class 3: 0.9236

    class 4: 0.935

    class 5: 0.9321

    class 6: 0.9566

    class 7: 0.9466

    class 8: 0.9608

    class 9: 0.9525

f1-score for each class:

    class 0: 0.533

    class 1: 0.5828

    class 2: 0.3768

    class 3: 0.3234

    class 4: 0.4253

    class 5: 0.4032

    class 6: 0.5538

    class 7: 0.5292

    class 8: 0.606

    class 9: 0.5487


Accuracy: 0.5147

Sensitivity for each class:

    class 0: 0.5924

    class 1: 0.5866

    class 2: 0.4074

    class 3: 0.3841

    class 4: 0.4449

    class 5: 0.4618

    class 6: 0.5292

    class 7: 0.5564

    class 8: 0.6059

    class 9: 0.5447

Specificity f

In [31]:
'''
The architecture of the improved models
adding 1 convolutional layer with 50 kernels sized 3x3 each.
adding 1 maxpool layer with kernel size 2x2
adding 1 fully connected layer with 50 neurons
'''

FLATTEN_CONVOLUTIONAL_SIZE_2 = 4*4*50

class improved_convolutional_layer(nn.Module):
    def __init__(self):
        super(improved_convolutional_layer, self).__init__()
        self.convolution1 = nn.Conv2d(3, 25, 3, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.convolution2 = nn.Conv2d(25, 50, 3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.convolution3 = nn.Conv2d(50, 50, 3, stride=1, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.linear1 = nn.Linear(FLATTEN_CONVOLUTIONAL_SIZE_2, 100)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(100, 50)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(50, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.convolution1(x)
        x = self.maxpool1(x)
        x = self.convolution2(x)
        x = self.maxpool2(x)
        x = self.convolution3(x)
        x = self.maxpool3(x)
        x = x.view(-1, FLATTEN_CONVOLUTIONAL_SIZE_2)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.softmax(x)
        
        return x

In [32]:
'''
Train the model with SGD and cross entropy loss function
use cuda interface if GPU exist
'''
model_cnn_imp =  improved_convolutional_layer()

if is_cuda:
    model_cnn_imp = model_cnn.cuda(cuda)
    
criterion_cnn_imp = nn.CrossEntropyLoss()
optimizer_cnn_imp = optim.SGD(model_cnn_imp.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCH):
    train_model(model_cnn_imp, train_loader, optimizer_cnn_imp, criterion_cnn_imp, epoch)

Epoch: 0

Accuracy: 0.5144

Sensitivity for each class:

    class 0: 0.5821

    class 1: 0.5884

    class 2: 0.4217

    class 3: 0.3813

    class 4: 0.4698

    class 5: 0.4715

    class 6: 0.5029

    class 7: 0.5195

    class 8: 0.6105

    class 9: 0.5461

Specificity for each class:

    class 0: 0.9493

    class 1: 0.9563

    class 2: 0.9306

    class 3: 0.9271

    class 4: 0.934

    class 5: 0.9347

    class 6: 0.9594

    class 7: 0.9521

    class 8: 0.9635

    class 9: 0.9552

f1-score for each class:

    class 0: 0.5604

    class 1: 0.5982

    class 2: 0.3919

    class 3: 0.3566

    class 4: 0.4294

    class 5: 0.4345

    class 6: 0.5654

    class 7: 0.5453

    class 8: 0.6411

    class 9: 0.5722


Epoch: 5

Accuracy: 0.5269

Sensitivity for each class:

    class 0: 0.585

    class 1: 0.6034

    class 2: 0.4375

    class 3: 0.3933

    class 4: 0.4766

    class 5: 0.4826

    class 6: 0.5201

    class 7: 0.5388

    class 8: 0.6219

    class 9: 

In [33]:
#save and test the model
file_location_model_cnn_imp = "model_cnn_imp.pt"
torch.save(model_cnn_imp.state_dict(), file_location_model_cnn_imp)

test_model(model_cnn_imp, test_loader)
test_model(model_cnn_imp, train_loader_2)

Accuracy: 0.5644

Sensitivity for each class:

    class 0: 0.5992

    class 1: 0.6865

    class 2: 0.5014

    class 3: 0.3926

    class 4: 0.4941

    class 5: 0.4756

    class 6: 0.6257

    class 7: 0.5766

    class 8: 0.6834

    class 9: 0.5945

Specificity for each class:

    class 0: 0.9588

    class 1: 0.9621

    class 2: 0.9298

    class 3: 0.9354

    class 4: 0.945

    class 5: 0.9392

    class 6: 0.9643

    class 7: 0.9571

    class 8: 0.9645

    class 9: 0.9608

f1-score for each class:

    class 0: 0.6147

    class 1: 0.6714

    class 2: 0.4102

    class 3: 0.4077

    class 4: 0.5

    class 5: 0.4619

    class 6: 0.6526

    class 7: 0.5961

    class 8: 0.6817

    class 9: 0.6215


Accuracy: 0.6351

Sensitivity for each class:

    class 0: 0.6572

    class 1: 0.7341

    class 2: 0.5862

    class 3: 0.4867

    class 4: 0.5367

    class 5: 0.5795

    class 6: 0.6801

    class 7: 0.6518

    class 8: 0.7633

    class 9: 0.6706

Specificity fo

In [34]:
'''
Train the model with bigger epoch. Epoch equals to 100
Train the model with Adam optimator with learning rate 0.0002
use cuda interface if GPU exist
'''
model_cnn_ht =  convolutional_layer()

if is_cuda:
    model_cnn_ht = model_cnn_ht.cuda(cuda)
    
#change optimator and its parameter
#using the same criterion function
lr = 0.0002
criterion_cnn_ht = nn.CrossEntropyLoss()
optimizer_cnn_ht = optim.Adam(model_cnn_ht.parameters(), lr)

for epoch in range(EPOCH_2):
    train_model(model_cnn_ht, train_loader, optimizer_cnn_ht, criterion_cnn_ht, epoch)

Epoch: 0

Accuracy: 0.3523

Sensitivity for each class:

    class 0: 0.454

    class 1: 0.324

    class 2: 0.3166

    class 3: 0.2845

    class 4: 0.3614

    class 5: 0.3657

    class 6: 0.2868

    class 7: 0.3654

    class 8: 0.4496

    class 9: 0.3422

Specificity for each class:

    class 0: 0.9318

    class 1: 0.9326

    class 2: 0.9127

    class 3: 0.9075

    class 4: 0.9183

    class 5: 0.9237

    class 6: 0.9441

    class 7: 0.929

    class 8: 0.9437

    class 9: 0.9422

f1-score for each class:

    class 0: 0.4101

    class 1: 0.3626

    class 2: 0.2257

    class 3: 0.1595

    class 4: 0.2858

    class 5: 0.3295

    class 6: 0.3764

    class 7: 0.3624

    class 8: 0.4734

    class 9: 0.4089


Epoch: 5

Accuracy: 0.5449

Sensitivity for each class:

    class 0: 0.5945

    class 1: 0.6226

    class 2: 0.4636

    class 3: 0.4084

    class 4: 0.4969

    class 5: 0.5024

    class 6: 0.5613

    class 7: 0.5581

    class 8: 0.6341

    class 9: 0

In [35]:
#save and test the model
file_location_model_cnn_ht = "model_cnn_ht.pt"
torch.save(model_cnn_ht.state_dict(), file_location_model_cnn_ht)

test_model(model_cnn_ht, test_loader)
test_model(model_cnn_ht, train_loader_2)

Accuracy: 0.657

Sensitivity for each class:

    class 0: 0.6704

    class 1: 0.7357

    class 2: 0.5827

    class 3: 0.5036

    class 4: 0.5807

    class 5: 0.5618

    class 6: 0.7295

    class 7: 0.7245

    class 8: 0.8093

    class 9: 0.6743

Specificity for each class:

    class 0: 0.9693

    class 1: 0.9747

    class 2: 0.9439

    class 3: 0.9436

    class 4: 0.9588

    class 5: 0.9529

    class 6: 0.9701

    class 7: 0.9678

    class 8: 0.9707

    class 9: 0.9674

f1-score for each class:

    class 0: 0.6971

    class 1: 0.7544

    class 2: 0.53

    class 3: 0.4972

    class 4: 0.6057

    class 5: 0.5693

    class 6: 0.7303

    class 7: 0.7172

    class 8: 0.7698

    class 9: 0.6907


Accuracy: 0.8446

Sensitivity for each class:

    class 0: 0.8541

    class 1: 0.8832

    class 2: 0.8173

    class 3: 0.7674

    class 4: 0.8135

    class 5: 0.8041

    class 6: 0.8586

    class 7: 0.8728

    class 8: 0.9016

    class 9: 0.871

Specificity fo